<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_deeplearning_python/blob/main/cnn_%EC%9D%8C%EC%95%85%EC%9E%A5%EB%A5%B4_%EB%B6%84%EB%A5%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
DATA_PATH="/content/drive/MyDrive/오디오_딥러닝_파이썬/result/data_10.json"


In [ ]:
def load_data(data_path):
  with open(data_path,"r") as fp:#json파일 읽어옴
    data=json.load(fp)

  X=np.array(data["mfcc"])#각 라벨별 데이터 가져와서 x,y에 넣음
  y=np.array(data["labels"])
  return X,y



In [ ]:
def plot_history(history):#학습 경과 그래프 그리기
  fig,axs=plt.subplot(2)

  axs[0].plot(history.history["accuracy"], label="train accuracy")
  axs[0].plot(history.history["val_accuracy"], label="test accuracy")
  axs[0].set_ylabel("Accuracy")
  axs[0].legend(loc="lower right")
  axs[0].set_title("Accuracy eval")

    # create error sublpot
  axs[1].plot(history.history["loss"], label="train error")
  axs[1].plot(history.history["val_loss"], label="test error")
  axs[1].set_ylabel("Error")
  axs[1].set_xlabel("Epoch")
  axs[1].legend(loc="upper right")
  axs[1].set_title("Error eval")

  plt.show()

In [ ]:
def prepare_datasets(test_size,validation_size):
  X,y=load_data(DATA_PATH)#데이터 읽어오기

  X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=test_size)
  #학습셋, 테스트셋 사이즈 크기로 분리
  X_train,X_validation,y_train,y_validation=train_test_split(X_train,y_train,test_size=validation_size)
  #이미 나눠진 트레인 셋에서 평가셋 분리, mfcc값을 학습셋, 테스트셋으로 분리함
  X_train=X_train[...,np.newaxis]
  # 학습용 mfcc값에 축 추가
  X_validation=X_validation[...,np.newaxis]
  # 평가용 mfcc값에 축 추가
  X_test=X_test[...,np.newaxis]
  # 테스트용 mfcc값에 축 추가

  return X_train,X_validation,X_test,y_train,y_validation,y_test

In [ ]:
def build_model(input_shape):
  #input_shape는 데이터에 따라서 달라짐, 이미지 100*100은 100*100*rgb(3)
  model=keras.Sequential()

  model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape))
  # 출력 채널 32개, 커널크기 3*3
  model.add(keras.layers.MaxPooling2D((3,3),strides=(2,2),padding='same'))
  # 풀링 층 추가, padding = same는 주변 패딩 추가해서 크기 유지
  model.add(keras.layers.BatchNormalization())# 해당 데이터 값 정규화(퀀타이징)

  model.add(keras.layers.Conv2D(32,(3,3),activation='relu'))
  model.add(keras.layers.MaxPooling2D((3,3),strides=(2,2),padding='same'))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Conv2D(32,(2,2),activation='relu'))
  #필터크기 2*2로 변경해서 작게 만듦
  model.add(keras.layers.MaxPooling2D((3,3),strides=(2,2),padding='same'))
  model.add(keras.layers.BatchNormalization())

  model.add(keras.layers.Flatten())
  #차원 줄임
  model.add(keras.layers.Dense(64,activation='relu'))
  model.add(keras.layers.Dropout(0.3))
  #드롭아웃
  model.add(keras.layers.Dense(10,activation='softmax'))
  #소프트맥스 활성화함수 지정
  return model

In [ ]:
def predict(model,X,y):
  X=X[np.newaxis,...]

  prediction=model.predict(X)

  predicted_index=np.argmax(prediction,axis=1)
  print("Target: {}, Predicted label: {}".format(y, predicted_index))

In [ ]:
if __name__=="__main__":
  #학습셋, 평가셋 생성
  X_train,X_validation,X_test,y_train,y_validation,y_test=prepare_datasets(0.25,0.2)#테스트셋 비율, 평가셋 비율
  input_shape=(X_train.shape[1],X_train.shape[2],1)#학습에 사용한 값 지정, mfcc값?
  model=build_model(input_shape)
  #모델 생성
  optimiser=keras.optimizers.Adam(learning_rate=0.0001)
  #
  model.compile(optimizer=optimiser,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  model.summary()

  history=model.fit(X_train,y_train,validation_data=(X_validation,y_validation),batch_size=32,epochs=30)
  #최적화 과정
  plot_history(history)

  test_loss,test_acc=model.evaluate(X_test,y_test,verbose=2)
  #차이값, 정확도 평가
  print('\nTest accuracy:', test_acc)

  X_to_predict=X_test[100]
  y_to_predict=y_test[100]

  predict(model,X_to_predict,y_to_predict)
  #예측

ValueError: With n_samples=0, test_size=0.25 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.